In [577]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directorye

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [578]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [579]:
train.dtypes

In [580]:
train.dtypes.value_counts().sum()

In [581]:
test.dtypes.value_counts().sum()

In [582]:
train.head()

In [583]:
test.head()

In [584]:
ytrain = train['SalePrice']

In [585]:
import matplotlib.pyplot as plt
import seaborn as sns

In [586]:
fig = plt.figure(figsize = (10,10) )
ax = fig.subplots(1,1)
ax.scatter(train['GarageArea'],ytrain)
ax.set_xlabel('GarageArea')
ax.set_ylabel('SalePrice')
plt.show()

In [587]:
corr = train.corr()
corr
fx,ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr)

In [588]:
Sale_relation = pd.DataFrame(corr['SalePrice'])
Sale_relation = Sale_relation.sort_values(by = 'SalePrice',ascending = False)
Sale_relation

In [589]:
top10 = Sale_relation.index[:11]

In [590]:
top10

In [591]:
top10data = train[top10]
top10corr = top10data.corr()
f,ax = plt.subplots(figsize = (10,10)) 
#sns.set(font_scale=1.25) 
sns.heatmap(top10corr,  annot=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=top10data.columns, xticklabels=top10data.columns)

In [592]:
sns.pairplot(top10data[top10[:5]],size=2.5)

In [593]:
top10test = test[top10[1:]]

In [594]:
train.isnull().sum().sort_values(ascending=False)

In [595]:
from sklearn.preprocessing import StandardScaler
scaled = top10data[:]
scaled['SalePrice'] = StandardScaler().fit_transform(scaled['SalePrice'][:,np.newaxis])

In [596]:
top10data.drop(['1stFlrSF','FullBath','TotRmsAbvGrd','YearRemodAdd','GarageArea'],1,inplace=True)
top10test.drop(['1stFlrSF','FullBath','TotRmsAbvGrd','YearRemodAdd','GarageArea'],1,inplace=True)

In [597]:
top10test['Id'] = test['Id']
top10test

In [598]:
test.loc[test['Id']==1461]['OverallQual']

In [599]:
top10data['Id'] = train['Id']
top10data

In [600]:
from scipy.stats import norm
sns.distplot((top10data['SalePrice']),fit=norm)

In [601]:
top10data['SalePrice']=np.log(top10data['SalePrice'])
ytrain = top10data['SalePrice']
top10data.drop('SalePrice',1,inplace = True)

In [602]:
top10test

In [603]:
sns.distplot(ytrain,fit=norm)

In [604]:
sns.distplot(top10data['GrLivArea'],fit=norm)

In [605]:
top10data['GrLivArea'] = np.log(top10data['GrLivArea'])
top10test['GrLivArea'] = np.log(top10data['GrLivArea'])

In [606]:
sns.distplot(top10data['GrLivArea'],fit=norm)

In [607]:
sns.distplot(top10data['TotalBsmtSF'], fit=norm);

In [608]:
top10data['hasbsmt'] = 0
top10test['hasbsmt'] = 0
top10data

In [609]:
top10data.loc[top10data['TotalBsmtSF']>0.0,'hasbsmt'] = 1 
top10test.loc[top10test['TotalBsmtSF']>0.0,'hasbsmt'] = 1 

In [610]:
top10test

In [611]:
top10data.loc[top10data['hasbsmt']==1,'TotalBsmtSF']=np.log(top10data['TotalBsmtSF'])
top10test.loc[top10test['hasbsmt']==1,'TotalBsmtSF']=np.log(top10test['TotalBsmtSF'])

In [612]:
sns.distplot(top10data[top10data['TotalBsmtSF']>0]['TotalBsmtSF'],fit=norm)

In [613]:
top10test

In [614]:
full = pd.concat([top10data,top10test])

In [615]:
catfull = pd.concat([full,pd.get_dummies(full['YearBuilt'])],1)

In [616]:
catfull = pd.concat([catfull,pd.get_dummies(full['OverallQual'])],1)

In [617]:
catfull = pd.concat([catfull,pd.get_dummies(full['GarageCars'])],1)

In [618]:
catfull.drop(['GarageCars','YearBuilt','OverallQual'],inplace=True,axis=1)

In [619]:
cat_train = catfull[:1460]

In [620]:
catfull

In [621]:
cat_test = catfull[1460:]

In [622]:
train.shape

In [623]:
cat_test

In [624]:
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split

In [625]:
ytrain.shape

In [626]:
x_train,x_cross,y_train,y_cross = train_test_split(cat_train,ytrain,test_size=0.10,random_state=42)

In [627]:
x_train.head()

In [628]:
model = rf()
model.fit(x_train,pd.DataFrame(y_train))

In [629]:
y_obt = model.predict(x_cross)

In [630]:
from sklearn.metrics import average_precision_score as pscr
from sklearn.metrics import r2_score as ascr
ascr(y_cross,y_obt)

In [631]:
xtrainid = cat_train['Id']
xtestid = cat_test['Id']

In [632]:
model.fit(cat_train,ytrain)

In [633]:
cat_test[cat_test['TotalBsmtSF'].isnull()]

In [634]:
cat_test['TotalBsmtSF'] = cat_test['TotalBsmtSF'].fillna(6.714643)

In [635]:
model.fit(cat_train,ytrain)

In [636]:
ypredtest = model.predict(cat_test)

In [637]:
lamb = lambda x: np.power(np.e,x)
ypredtest = np.array([lamb(i) for i in ypredtest])

In [638]:
cat_test

In [639]:
submission = pd.DataFrame({'Id':xtestid, 'SalePrice':ypredtest})

In [640]:
submission

In [641]:
submission.to_csv('submission1.csv',index=False)